In [1]:
import pandas as pd

In [2]:
file_path_episodes = "../exploration/BIG_DF_ML.ipynb"

db01 = pd.read_csv("../gitignore/title_basics_traite.csv")
db02 = pd.read_csv("../gitignore/title_ratings_final.tsv", sep="\t")
db03 = pd.read_csv("../gitignore/title.akas_final.tsv", sep="\t")
db04 = pd.read_csv("../gitignore/tmdb_ml_final.csv")
db05 = pd.read_csv("../gitignore/data_bechdel.csv")
#db06 = pd.read_csv("../gitignore/title.principals4.tsv", sep="\t") #INFO
#db07 = pd.read_csv("../gitignore/name.basics.tsv", sep="\t") # INFO
#db08 = pd.read_csv("../gitignore/title.crew.tsv", sep="\t") # INFO

In [3]:
db01_tot = db01.shape[0] # TITLE BASICS
print(f"Total: {db01_tot}")
db01.nunique()

Total: 688341


tconst            688341
titleType              1
startYear            138
runtimeMinutes       517
genres              1398
decade                16
Action                 2
Adult                  1
Adventure              2
Animation              2
Biography              2
Comedy                 2
Crime                  2
Documentary            2
Drama                  2
Family                 2
Fantasy                2
Game-Show              2
History                2
Horror                 2
Music                  2
Musical                2
Mystery                2
News                   2
Reality-TV             2
Romance                2
Sci-Fi                 2
Short                  1
Sport                  2
Talk-Show              2
Thriller               2
War                    2
Western                2
movie                  1
dtype: int64

In [4]:
display(db01.sample(10))

,tconst,titleType,startYear,runtimeMinutes,genres,decade,Action,Adult,Adventure,Animation,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,movie
44298,tt0060736,movie,1965,96,"Adventure,Drama,Thriller",1960,False,False,True,False,...,False,False,False,False,False,False,True,False,False,True
5320,tt0011577,movie,1920,200,"Action,Adventure",1920,True,False,True,False,...,False,False,False,False,False,False,False,False,False,True
13899,tt0022049,movie,1932,0,\N,1930,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
570349,tt4624184,movie,2015,56,Documentary,2010,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
391525,tt2076315,movie,2011,0,Romance,2010,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
98707,tt0163419,movie,1996,115,\N,1990,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
245620,tt11111286,movie,2020,97,Comedy,2020,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
672261,tt8998236,movie,2018,74,Documentary,2010,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
62032,tt0087146,movie,1984,0,Drama,1980,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
345003,tt15856932,movie,2021,90,"Comedy,Crime,Drama",2020,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [5]:
db02_tot = db02.shape[0] # TITLE RATINGS
print(f"Total: {db02_tot}")
db02.nunique()

Total: 1500950


title_ratings_tconst           1500950
title_ratings_averageRating         91
title_ratings_numVotes           22785
dtype: int64

In [6]:
dbmerge_1 = pd.merge(db01, db02, right_on='title_ratings_tconst', left_on='tconst', how='left') # TITLE BASICS + TITLE RATINGS

In [7]:
display(dbmerge_1.head()) # TITLE BASICS + TITLE RATINGS

,tconst,titleType,startYear,runtimeMinutes,genres,decade,Action,Adult,Adventure,Animation,...,Short,Sport,Talk-Show,Thriller,War,Western,movie,title_ratings_tconst,title_ratings_averageRating,title_ratings_numVotes
0,tt0000009,movie,1894,45,Romance,1890,False,False,False,False,...,False,False,False,False,False,False,True,tt0000009,5.4,215.0
1,tt0000147,movie,1897,100,"Documentary,News,Sport",1890,False,False,False,False,...,False,True,False,False,False,False,True,tt0000147,5.2,539.0
2,tt0000502,movie,1905,100,\N,1900,False,False,False,False,...,False,False,False,False,False,False,True,tt0000502,4.4,18.0
3,tt0000574,movie,1906,70,"Action,Adventure,Biography",1900,True,False,True,False,...,False,False,False,False,False,False,True,tt0000574,6.0,940.0
4,tt0000591,movie,1907,90,Drama,1900,False,False,False,False,...,False,False,False,False,False,False,True,tt0000591,5.7,28.0


In [8]:
dbmerge_1_tot = dbmerge_1.shape[0] # TITLE BASICS + TITLE RATINGS
print(f"Total: {dbmerge_1_tot}")
dbmerge_1.nunique()

Total: 688341


tconst                         688341
titleType                           1
startYear                         138
runtimeMinutes                    517
genres                           1398
decade                             16
Action                              2
Adult                               1
Adventure                           2
Animation                           2
Biography                           2
Comedy                              2
Crime                               2
Documentary                         2
Drama                               2
Family                              2
Fantasy                             2
Game-Show                           2
History                             2
Horror                              2
Music                               2
Musical                             2
Mystery                             2
News                                2
Reality-TV                          2
Romance                             2
Sci-Fi      

In [9]:
db03_tot = db03.shape[0] # TITLE AKAS
print(f"Total: {db03_tot}")
db03.nunique()

Total: 11214224


titleId            11214224
ordering                 78
title               5073850
region                    2
language                  5
types                     9
attributes               91
isOriginalTitle           2
liste_titles        5076704
dtype: int64

In [10]:
display(db03.sample(10))

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle,liste_titles
4185251,tt6309106,4,Épisode #1.1184,FR,fr,\N,\N,0,['Épisode #1.1184']
7141733,tt15433910,1,Introducing Hayli Sanders,\N,\N,original,\N,1,['Introducing Hayli Sanders']
7503470,tt17052750,1,Enfant disparu,\N,\N,original,\N,1,['Enfant disparu']
10318901,tt6338414,1,"Kanye's Legal, Financial and Health Crisis",\N,\N,original,\N,1,"[""Kanye's Legal, Financial and Health Crisis""]"
6685599,tt1377453,1,Dinner with Dad,\N,\N,original,\N,1,['Dinner with Dad']
2589866,tt2311771,4,Épisode datant du 1 octobre 2001,FR,fr,\N,\N,0,['Épisode datant du 1 octobre 2001']
5989513,tt11141850,1,La Peur,\N,\N,original,\N,1,['La Peur']
9485777,tt3449466,1,Lo mismo,\N,\N,original,\N,1,['Lo mismo']
1182387,tt13456560,4,Épisode datant du 3 décembre 2015,FR,fr,\N,\N,0,['Épisode datant du 3 décembre 2015']
3621162,tt33341587,4,Épisode #1.532,FR,fr,\N,\N,0,['Épisode #1.532']


In [11]:
dbmerge_2 = pd.merge(dbmerge_1, db03, left_on='tconst', right_on='titleId', how='left') # (TITLE BASICS + TITLE RATINGS) + TITLE AKAS

In [12]:
display(dbmerge_2.head(10)) # (TITLE BASICS + TITLE RATINGS) + TITLE AKAS

,tconst,titleType,startYear,runtimeMinutes,genres,decade,Action,Adult,Adventure,Animation,...,title_ratings_numVotes,titleId,ordering,title,region,language,types,attributes,isOriginalTitle,liste_titles
0,tt0000009,movie,1894,45,Romance,1890,False,False,False,False,...,215.0,tt0000009,1.0,Miss Jerry,\N,\N,original,\N,1.0,['Miss Jerry']
1,tt0000147,movie,1897,100,"Documentary,News,Sport",1890,False,False,False,False,...,539.0,tt0000147,1.0,The Corbett-Fitzsimmons Fight,\N,\N,original,\N,1.0,['The Corbett-Fitzsimmons Fight']
2,tt0000502,movie,1905,100,\N,1900,False,False,False,False,...,18.0,tt0000502,1.0,Bohemios,\N,\N,original,\N,1.0,['Bohemios']
3,tt0000574,movie,1906,70,"Action,Adventure,Biography",1900,True,False,True,False,...,940.0,tt0000574,1.0,The Story of the Kelly Gang,\N,\N,original,\N,1.0,['The Story of the Kelly Gang']
4,tt0000591,movie,1907,90,Drama,1900,False,False,False,False,...,28.0,tt0000591,2.0,L'enfant prodigue,FR,\N,imdbDisplay,\N,0.0,"[""L'enfant prodigue""]"
5,tt0000615,movie,1907,0,Drama,1900,False,False,False,False,...,27.0,tt0000615,1.0,Robbery Under Arms,\N,\N,original,\N,1.0,['Robbery Under Arms']
6,tt0000630,movie,1908,0,Drama,1900,False,False,False,False,...,30.0,tt0000630,1.0,Amleto,\N,\N,original,\N,1.0,['Amleto']
7,tt0000675,movie,1908,0,Drama,1900,False,False,False,False,...,22.0,tt0000675,1.0,Don Quijote,\N,\N,original,\N,1.0,['Don Quijote']
8,tt0000679,movie,1908,120,"Adventure,Fantasy",1900,False,False,True,False,...,78.0,tt0000679,1.0,The Fairylogue and Radio-Plays,\N,\N,original,\N,1.0,['The Fairylogue and Radio-Plays']
9,tt0000838,movie,1909,0,\N,1900,False,False,False,False,...,NaN,tt0000838,1.0,A Cultura do Cacau,\N,\N,original,\N,1.0,['A Cultura do Cacau']


In [13]:
dbmerge_2_tot = dbmerge_2.shape[0] # (TITLE BASICS + TITLE RATINGS) + TITLE AKAS
print(f"Total: {dbmerge_2_tot}")
dbmerge_2.nunique()

Total: 688341


tconst                         688341
titleType                           1
startYear                         138
runtimeMinutes                    517
genres                           1398
decade                             16
Action                              2
Adult                               1
Adventure                           2
Animation                           2
Biography                           2
Comedy                              2
Crime                               2
Documentary                         2
Drama                               2
Family                              2
Fantasy                             2
Game-Show                           2
History                             2
Horror                              2
Music                               2
Musical                             2
Mystery                             2
News                                2
Reality-TV                          2
Romance                             2
Sci-Fi      

In [14]:
db04_tot = db04.shape[0] # TMDB FULL
print(f"Total: {db04_tot}")
db04.nunique()

Total: 306290


tmdb_imdb_id           306290
tmdb_original_title    281936
tmdb_popularity         18234
tmdb_release_date         141
tmdb_runtime              460
                        ...  
tmdb_IR                     2
tmdb_XG                     2
tmdb_EG                     2
tmdb_NG                     2
tmdb_ZA                     2
Length: 62, dtype: int64

In [15]:
dbmerge_3 = pd.merge(dbmerge_2, db04, left_on='tconst', right_on='tmdb_imdb_id', how='left') # ((TITLE BASICS + TITLE RATINGS) + TITLE AKAS + TMDB FULL)

In [16]:
display(dbmerge_3.head()) # ((TITLE BASICS + TITLE RATINGS) + TITLE AKAS + TMDB FULL)

,tconst,titleType,startYear,runtimeMinutes,genres,decade,Action,Adult,Adventure,Animation,...,tmdb_PL,tmdb_CH,tmdb_XC,tmdb_FI,tmdb_NO,tmdb_IR,tmdb_XG,tmdb_EG,tmdb_NG,tmdb_ZA
0,tt0000009,movie,1894,45,Romance,1890,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0000147,movie,1897,100,"Documentary,News,Sport",1890,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,tt0000502,movie,1905,100,\N,1900,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0000574,movie,1906,70,"Action,Adventure,Biography",1900,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,tt0000591,movie,1907,90,Drama,1900,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
dbmerge_3_tot = dbmerge_3.shape[0] # ((TITLE BASICS + TITLE RATINGS) + TITLE AKAS + TMDB FULL)
print(f"Total: {dbmerge_3_tot}")
dbmerge_3.nunique()

Total: 688341


tconst            688341
titleType              1
startYear            138
runtimeMinutes       517
genres              1398
                   ...  
tmdb_IR                2
tmdb_XG                2
tmdb_EG                2
tmdb_NG                2
tmdb_ZA                2
Length: 108, dtype: int64

In [18]:
db05_tot = db05.shape[0] # BECHDEL
print(f"Total: {db05_tot}")
db05.nunique()

Total: 10440


rating        4
imdbid    10440
dtype: int64

In [19]:
dbmerge_4 = pd.merge(dbmerge_3, db05, left_on='tconst', right_on='imdbid', how='left') # (((TITLE BASICS + TITLE RATINGS) + TITLE AKAS + TMDB FULL) + BECHDEL)

In [20]:
display(dbmerge_4.head()) # (((TITLE BASICS + TITLE RATINGS) + TITLE AKAS + TMDB FULL) + BECHDEL)

,tconst,titleType,startYear,runtimeMinutes,genres,decade,Action,Adult,Adventure,Animation,...,tmdb_XC,tmdb_FI,tmdb_NO,tmdb_IR,tmdb_XG,tmdb_EG,tmdb_NG,tmdb_ZA,rating,imdbid
0,tt0000009,movie,1894,45,Romance,1890,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,tt0000009
1,tt0000147,movie,1897,100,"Documentary,News,Sport",1890,False,False,False,False,...,False,False,False,False,False,False,False,False,NaN,NaN
2,tt0000502,movie,1905,100,\N,1900,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0000574,movie,1906,70,"Action,Adventure,Biography",1900,True,False,True,False,...,False,False,False,False,False,False,False,False,1.0,tt0000574
4,tt0000591,movie,1907,90,Drama,1900,False,False,False,False,...,False,False,False,False,False,False,False,False,NaN,NaN


In [21]:
dbmerge_4_tot = dbmerge_4.shape[0] # (((TITLE BASICS + TITLE RATINGS) + TITLE AKAS + TMDB FULL) + BECHDEL) = BIG_DF_RAW
print(f"Total: {dbmerge_4_tot}")
dbmerge_4.nunique()

Total: 688341


tconst            688341
titleType              1
startYear            138
runtimeMinutes       517
genres              1398
                   ...  
tmdb_EG                2
tmdb_NG                2
tmdb_ZA                2
rating                 4
imdbid              9621
Length: 110, dtype: int64

In [22]:
display(dbmerge_4.isna().sum())

tconst                 0
titleType              0
startYear              0
runtimeMinutes         0
genres                 0
                   ...  
tmdb_EG           485874
tmdb_NG           485874
tmdb_ZA           485874
rating            678720
imdbid            678720
Length: 110, dtype: int64

In [23]:
#dbmerge_4.to_csv("../gitignore/BIG_DF_ML.csv", index=False)

In [24]:
dbmerge_4['movie'].value_counts()

movie
True    688341
Name: count, dtype: int64

In [25]:
BIG_DF_ML = dbmerge_4.drop(columns=[
    'titleType',
    'genres', 
    'decade', 
    'Adult',
    'Short',
    'movie',
    'tmdb_Comedy',
    'tmdb_Adventure',
    'tmdb_Drama',
    'tmdb_Crime',
    'tmdb_Action',
    'tmdb_Documentary',
    'tmdb_Animation',
    'tmdb_Mystery',
    'tmdb_Horror',
    'tmdb_Western',
    'tmdb_Science Fiction',
    'tmdb_Thriller',
    'tmdb_Romance',
    'tmdb_Fantasy',
    'tmdb_Family',
    'tmdb_History',
    'tmdb_Music',
    'tmdb_War'
])

In [26]:
BIG_DF_ML.loc[(dbmerge_4['title_ratings_averageRating'].isnull())&(dbmerge_4['tmdb_vote_average'].isnull()), ['title_ratings_tconst', 'title_ratings_averageRating', 'tmdb_vote_average']]

,title_ratings_tconst,title_ratings_averageRating,tmdb_vote_average
9,NaN,NaN,NaN
10,NaN,NaN,NaN
11,NaN,NaN,NaN
12,NaN,NaN,NaN
13,NaN,NaN,NaN
...,...,...,...
688329,NaN,NaN,NaN
688335,NaN,NaN,NaN
688336,NaN,NaN,NaN
688337,NaN,NaN,NaN


In [27]:
BIG_DF_ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688341 entries, 0 to 688340
Data columns (total 86 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   tconst                       688341 non-null  object 
 1   startYear                    688341 non-null  int64  
 2   runtimeMinutes               688341 non-null  int64  
 3   Action                       688341 non-null  bool   
 4   Adventure                    688341 non-null  bool   
 5   Animation                    688341 non-null  bool   
 6   Biography                    688341 non-null  bool   
 7   Comedy                       688341 non-null  bool   
 8   Crime                        688341 non-null  bool   
 9   Documentary                  688341 non-null  bool   
 10  Drama                        688341 non-null  bool   
 11  Family                       688341 non-null  bool   
 12  Fantasy                      688341 non-null  bool   
 13 

In [28]:
BIG_DF_ML3 = BIG_DF_ML.dropna(subset=['title_ratings_averageRating','tmdb_vote_average'],how='all')

In [29]:
BIG_DF_ML3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331142 entries, 0 to 688339
Data columns (total 86 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   tconst                       331142 non-null  object 
 1   startYear                    331142 non-null  int64  
 2   runtimeMinutes               331142 non-null  int64  
 3   Action                       331142 non-null  bool   
 4   Adventure                    331142 non-null  bool   
 5   Animation                    331142 non-null  bool   
 6   Biography                    331142 non-null  bool   
 7   Comedy                       331142 non-null  bool   
 8   Crime                        331142 non-null  bool   
 9   Documentary                  331142 non-null  bool   
 10  Drama                        331142 non-null  bool   
 11  Family                       331142 non-null  bool   
 12  Fantasy                      331142 non-null  bool   
 13  Game

In [30]:
def moyenne_ponderee(ligne):

    # Si 'title_ratings_averageRating' est NaN, on ne prend que 'tmdb_vote_average', et vice versa

    if pd.isna(ligne['title_ratings_averageRating']) and not pd.isna(ligne['tmdb_vote_average']):
        return ligne['tmdb_vote_average']  # Si title_ratings_averageRating est vide, prendre tmdb_vote_average

    elif pd.isna(ligne['tmdb_vote_average']) and not pd.isna(ligne['title_ratings_averageRating']):
        return ligne['title_ratings_averageRating']  # Si tmdb_vote_average est vide, prendre title_ratings_averageRating

    elif not pd.isna(ligne['title_ratings_averageRating']) and not pd.isna(ligne['tmdb_vote_average']):
        # Si les deux colonnes ont des valeurs, calculer la moyenne pondérée
        return (ligne['title_ratings_averageRating'] * ligne['title_ratings_numVotes'] + ligne['tmdb_vote_average'] * ligne['tmdb_vote_count']) / (ligne['title_ratings_numVotes'] + ligne['tmdb_vote_count'])  # Moyenne simple, à ajuster si besoin !
    else:
        return np.nan  # Si les deux sont NaN, retourner NaN


BIG_DF_ML3['notes'] = BIG_DF_ML3.apply(moyenne_ponderee, axis=1)

C:\Users\frale\AppData\Local\Temp\ipykernel_24852\2696331554.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BIG_DF_ML3['notes'] = BIG_DF_ML3.apply(moyenne_ponderee, axis=1)


In [31]:
BIG_DF_ML3[['notes','title_ratings_averageRating','tmdb_vote_average']].sample(10)

,notes,title_ratings_averageRating,tmdb_vote_average
255937,5.000000,NaN,5.000
100013,7.100000,7.1,NaN
231356,6.300000,6.3,NaN
55289,6.899331,6.9,6.841
623069,8.300000,8.3,NaN
87074,6.200000,6.2,NaN
289114,6.006771,6.0,6.500
123723,6.088235,6.1,5.800
248293,5.100000,5.1,0.000
35918,6.098762,6.1,6.000


In [32]:
BIG_DF_ML4 = BIG_DF_ML3.drop(['title_ratings_averageRating','tmdb_vote_average','title_ratings_tconst','titleId','tmdb_imdb_id','imdbid'], axis=1)

In [33]:
BIG_DF_ML4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331142 entries, 0 to 688339
Data columns (total 81 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   tconst                  331142 non-null  object 
 1   startYear               331142 non-null  int64  
 2   runtimeMinutes          331142 non-null  int64  
 3   Action                  331142 non-null  bool   
 4   Adventure               331142 non-null  bool   
 5   Animation               331142 non-null  bool   
 6   Biography               331142 non-null  bool   
 7   Comedy                  331142 non-null  bool   
 8   Crime                   331142 non-null  bool   
 9   Documentary             331142 non-null  bool   
 10  Drama                   331142 non-null  bool   
 11  Family                  331142 non-null  bool   
 12  Fantasy                 331142 non-null  bool   
 13  Game-Show               331142 non-null  bool   
 14  History                 3

In [34]:
def startyear(ligne):

    # Si 'start year' est 0, il prend la valeur 'tmbd release date'
    BIG_DF_ML4.loc[(BIG_DF_ML4['startYear'] == 0)&(BIG_DF_ML4['tmdb_release_date'] != 0), 'startYear'] = BIG_DF_ML4['tmdb_release_date']

startyear(BIG_DF_ML4)

In [35]:
def release_date(ligne):

    # Si 'tmdn release date' est 0, il prend la valeur 'start year'
    BIG_DF_ML4.loc[(BIG_DF_ML4['tmdb_release_date'] == 0)&(BIG_DF_ML4['startYear'] != 0), 'tmdb_release_date'] = BIG_DF_ML4['startYear']

release_date(BIG_DF_ML4)

In [36]:
def release_date(ligne):

    # Si 'tmdn release date' est 0, il prend la valeur 'start year'
    BIG_DF_ML4.loc[(BIG_DF_ML4['tmdb_release_date'].isna())&(BIG_DF_ML4['startYear'] != 0), 'tmdb_release_date'] = BIG_DF_ML4['startYear']

release_date(BIG_DF_ML4)

In [37]:
BIG_DF_ML4[BIG_DF_ML4['startYear']==0]['startYear']

203435    0.0
221678    0.0
225630    0.0
235096    0.0
235166    0.0
         ... 
671323    0.0
679878    0.0
680412    0.0
684006    0.0
685415    0.0
Name: startYear, Length: 200, dtype: float64

In [38]:
BIG_DF_ML4['tmdb_release_date'].isna().sum()

22

In [39]:
BIG_DF_ML4['startYear'].isna().sum()

22

In [40]:
def runtimeMinutes(ligne):

    # Si 'runtimeMinutes' est 0, il prend la valeur 'tmdb_runtime'
    BIG_DF_ML4.loc[(BIG_DF_ML4['runtimeMinutes'] == 0)&(BIG_DF_ML4['tmdb_runtime'] != 0), 'runtimeMinutes'] = BIG_DF_ML4['tmdb_runtime']

runtimeMinutes(BIG_DF_ML4)

In [41]:
def runtimeMinutes(ligne):

    # Si 'tmdb_runtime' est 0, il prend la valeur 'runtimeMinutes'
    BIG_DF_ML4.loc[(BIG_DF_ML4['tmdb_runtime'] == 0)&(BIG_DF_ML4['runtimeMinutes'] != 0), 'tmdb_runtime'] = BIG_DF_ML4['runtimeMinutes']

runtimeMinutes(BIG_DF_ML4)

In [42]:
BIG_DF_ML4[BIG_DF_ML4['runtimeMinutes']==0]['runtimeMinutes']

36        0.0
58        0.0
67        0.0
74        0.0
133       0.0
         ... 
686766    0.0
686949    0.0
686965    0.0
687577    0.0
687929    0.0
Name: runtimeMinutes, Length: 5623, dtype: float64

In [43]:
BIG_DF_ML4[BIG_DF_ML4['tmdb_runtime']==0]['tmdb_runtime']

36        0.0
58        0.0
67        0.0
74        0.0
133       0.0
         ... 
686766    0.0
686949    0.0
686965    0.0
687577    0.0
687929    0.0
Name: tmdb_runtime, Length: 5623, dtype: float64

In [44]:
BIG_DF_ML4['runtimeMinutes'].isna().sum()

24038

In [45]:
BIG_DF_ML4['tmdb_runtime'].isna().sum()

128675

In [46]:
#On transforme les valeurs 0 en NaN pour les supprimer ensuite
import numpy as np
BIG_DF_ML4[BIG_DF_ML4['startYear']==0] = np.nan

C:\Users\frale\AppData\Local\Temp\ipykernel_24852\2379037837.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  BIG_DF_ML4[BIG_DF_ML4['startYear']==0] = np.nan
C:\Users\frale\AppData\Local\Temp\ipykernel_24852\2379037837.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  BIG_DF_ML4[BIG_DF_ML4['startYear']==0] = np.nan
C:\Users\frale\AppData\Local\Temp\ipykernel_24852\2379037837.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  BIG_DF_ML4[BIG_DF_ML4['startYear']==0

In [47]:
#On transforme les valeurs 0 en NaN pour les supprimer ensuite
BIG_DF_ML4[BIG_DF_ML4['runtimeMinutes']==0] = np.nan

In [48]:
BIG_DF_ML5 = BIG_DF_ML4.drop(columns =['tmdb_runtime','tmdb_release_date','ordering','region','language','types','attributes','isOriginalTitle','tmdb_original_title','tmdb_title','tmdb_vote_count','tmdb_TV Movie'])

In [49]:
BIG_DF_ML5 = BIG_DF_ML5.dropna(subset=['runtimeMinutes'])

In [50]:
BIG_DF_ML5 = BIG_DF_ML5.dropna(subset=['startYear'])

In [51]:
BIG_DF_ML5 = BIG_DF_ML5.dropna(subset=['title'])

In [52]:
# Pour notre ML, on remplace toutes les valeurs nulles des pays de production par False 
Remplacer = [
    'tmdb_US', 'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
    'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE', 'tmdb_SU',
    'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR', 'tmdb_RU', 'tmdb_DK',
    'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR', 'tmdb_PL', 'tmdb_CH', 'tmdb_XC',
    'tmdb_FI', 'tmdb_NO', 'tmdb_IR', 'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'
]
BIG_DF_ML5[Remplacer] = BIG_DF_ML5[Remplacer].fillna(False)

C:\Users\frale\AppData\Local\Temp\ipykernel_24852\61637864.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  BIG_DF_ML5[Remplacer] = BIG_DF_ML5[Remplacer].fillna(False)


In [53]:
BIG_DF_ML5['rating'] = BIG_DF_ML5['rating'].fillna(0)

In [54]:
BIG_DF_ML5['title_ratings_numVotes'] = BIG_DF_ML5['title_ratings_numVotes'].fillna(0)

In [55]:
BIG_DF_ML5['tmdb_popularity'] = BIG_DF_ML5['tmdb_popularity'].fillna(0)

In [56]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

import seaborn as sns
import matplotlib.pyplot as plt

# Premier test de ML

In [77]:
def encodage_X(X, type='standard'):
  index = X.index
  X_num = X.select_dtypes('number')
  X_cat = X.select_dtypes(['object', 'category', 'string', 'bool'])

  if type == 'standard':
    from sklearn.preprocessing import StandardScaler
    SN = StandardScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  else:
    from sklearn.preprocessing import MinMaxScaler
    SN = MinMaxScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)


  X_encoded = pd.concat([X_num_SN, X_cat], axis=1)

  return X_encoded, SN

X_encoded, SN = encodage_X(BIG_DF_ML5, type = 'normal')

In [80]:
#On crée une liste des colonnes à utiliser pour le modèle
caracteristiques = X_encoded.columns.drop(['tconst', 'title', 'liste_titles'])
caracteristiques

In [79]:
model = NearestNeighbors(n_neighbors=5, metric='euclidean')
model.fit(X_encoded.drop(columns = ['tconst', 'title', 'liste_titles']))

NearestNeighbors(metric='euclidean')

In [81]:
caract_film = X_encoded[X_encoded['title']== 'Inception']
caract_film

,startYear,runtimeMinutes,title_ratings_numVotes,tmdb_popularity,rating,notes,tconst,Action,Adventure,Animation,...,tmdb_PL,tmdb_CH,tmdb_XC,tmdb_FI,tmdb_NO,tmdb_IR,tmdb_XG,tmdb_EG,tmdb_NG,tmdb_ZA
300525,0.885496,0.002472,0.881844,0.015148,1.0,0.879444,tt1375666,True,True,False,...,False,False,False,False,False,False,False,False,False,False


In [83]:
caract_film = caract_film[caracteristiques]
caract_film

,startYear,runtimeMinutes,title_ratings_numVotes,tmdb_popularity,rating,notes,Action,Adventure,Animation,Biography,...,tmdb_PL,tmdb_CH,tmdb_XC,tmdb_FI,tmdb_NO,tmdb_IR,tmdb_XG,tmdb_EG,tmdb_NG,tmdb_ZA
300525,0.885496,0.002472,0.881844,0.015148,1.0,0.879444,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [85]:
distances, indices = model.kneighbors(caract_film)

In [88]:
X_encoded.iloc[indices[0]]

,startYear,runtimeMinutes,title_ratings_numVotes,tmdb_popularity,rating,notes,tconst,Action,Adventure,Animation,...,tmdb_PL,tmdb_CH,tmdb_XC,tmdb_FI,tmdb_NO,tmdb_IR,tmdb_XG,tmdb_EG,tmdb_NG,tmdb_ZA
300525,0.885496,0.002472,0.881844,0.015148,1.0,0.879444,tt1375666,True,True,False,...,False,False,False,False,False,False,False,False,False,False
62787,0.687023,0.001783,0.318948,0.009709,1.0,0.809450,tt0088247,True,True,False,...,False,False,False,False,False,False,False,False,False,False
208782,0.908397,0.002388,0.277552,0.011870,1.0,0.709174,tt0770828,True,True,False,...,False,False,False,False,False,False,False,False,False,False
58480,0.656489,0.002119,0.039345,0.004318,1.0,0.679896,tt0081573,True,True,False,...,False,False,False,False,False,False,False,False,False,False
66472,0.709924,0.001497,0.018184,0.004291,1.0,0.371627,tt0094074,True,True,False,...,False,False,False,False,False,False,False,False,False,False


# Second test de ML avec cible sur les notes > 7

In [124]:
def encodage_X(X, type='standard'):
  index = X.index
  X_num = X.select_dtypes('number')
  X_cat = X.select_dtypes(['object', 'category', 'string', 'bool'])
 
  if type == 'standard':
    from sklearn.preprocessing import StandardScaler
    SN = StandardScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  else:
    from sklearn.preprocessing import MinMaxScaler
    SN = MinMaxScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  X_encoded = pd.concat([X_num_SN, X_cat], axis=1)

  return X_encoded, SN

In [125]:
X = BIG_DF_ML5
X_encoded, SN = encodage_X(X, type = 'normal')

In [132]:
bons_films = BIG_DF_ML5[BIG_DF_ML5['notes'] >= 0.7]
mauvais_films = BIG_DF_ML5[BIG_DF_ML5['notes'] < 0.7]

In [133]:
#On crée une liste des colonnes à utiliser pour le modèle
caracteristiques = X_encoded.columns.drop(['tconst', 'title', 'liste_titles'])
caracteristiques

Index(['startYear', 'runtimeMinutes', 'title_ratings_numVotes',
       'tmdb_popularity', 'rating', 'notes', 'Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Sport', 'Talk-Show', 'Thriller', 'War', 'Western', 'tmdb_US',
       'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
       'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE',
       'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR',
       'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR',
       'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO', 'tmdb_IR',
       'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'],
      dtype='object')

In [135]:
model = NearestNeighbors(n_neighbors=5, metric='euclidean')
model.fit(X_encoded.drop(columns=['tconst', 'title', 'liste_titles']))

NearestNeighbors(metric='euclidean')

In [136]:
def encodage_predict(df_a_predire):
  X_num = df_a_predire.select_dtypes('number')
  X_cat = df_a_predire.select_dtypes(['object', 'category', 'string', 'bool'])

  X_num_SN = pd.DataFrame(SN.transform(X_num), columns=X_num.columns).reset_index(drop=True)
  X_cat = X_cat.reset_index(drop=True)

  X_encoded_predire = pd.concat([X_num_SN, X_cat], axis=1)

  df_predict = X_encoded_predire

  # DataFrame vide qui a les mêmes colonnes que X_encoded
  df_final = pd.DataFrame(columns=X_encoded.columns)

  # On veut que le DataFrame ait le même nombre de lignes que df_predict
  df_final = df_final.reindex(index=df_predict.index)
  # On met tous les NaN à False
  df_final = df_final.fillna(False)

  # On parcourt chaque colonne de df_predict
  # Si la colonne est présente dans X_encoded alors on la garde
  # Sinon, on la met à False
  for column in df_predict.columns:
    if column in X_encoded.columns:
      df_final[column] = df_predict[column]

  return df_final

In [137]:
caract_film = X_encoded[X_encoded['title'] == 'Inception']
caract_film = X_encoded[caracteristiques]
caract_film


,startYear,runtimeMinutes,title_ratings_numVotes,tmdb_popularity,rating,notes,Action,Adventure,Animation,Biography,...,tmdb_PL,tmdb_CH,tmdb_XC,tmdb_FI,tmdb_NO,tmdb_IR,tmdb_XG,tmdb_EG,tmdb_NG,tmdb_ZA
0,0.000000,0.000740,0.000073,0.000000,0.000000,0.540000,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0.022901,0.001665,0.000182,0.000188,0.000000,0.521166,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,0.083969,0.001665,0.000006,0.000000,0.000000,0.440000,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,0.091603,0.001160,0.000317,0.000328,0.333333,0.598192,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False
4,0.099237,0.001497,0.000009,0.000118,0.000000,0.570000,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688330,0.961832,0.001581,0.000088,0.001203,0.000000,0.362205,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
688331,0.961832,0.001396,0.000503,0.001334,0.000000,0.592675,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
688332,0.961832,0.001530,0.002003,0.001695,0.000000,0.641277,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
688334,0.954198,0.002052,0.000003,0.000000,0.000000,0.830000,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [123]:
caract_film_cible = encodage_predict(caract_film)
caract_film_cible

,startYear,runtimeMinutes,title_ratings_numVotes,tmdb_popularity,rating,notes,Action,Adventure,Animation,Biography,...,tmdb_PL,tmdb_CH,tmdb_XC,tmdb_FI,tmdb_NO,tmdb_IR,tmdb_XG,tmdb_EG,tmdb_NG,tmdb_ZA
0,0.876106,0.002472,0.881844,0.019591,1.0,0.598146,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
distances, indices = model.kneighbors()

In [ ]:
distances = distances[1:]
indices = indices[1:]

# OPTIMISATION DE K

In [147]:
def evaluate_k(X_encoded, k_range):
    """
    Évalue différentes valeurs de k en utilisant la somme des distances aux voisins
    et le score de silhouette comme métriques.

    Args:
        X_encoded (DataFrame): Données normalisées
        k_range (range): Plage de valeurs de k à tester

    Returns:
        tuple: (distances moyennes, scores de silhouette)
    """
    from sklearn.metrics import silhouette_score
    from sklearn.cluster import KMeans
    from sklearn.neighbors import NearestNeighbors
    import numpy as np

    avg_distances = []
    silhouette_scores = []

    for k in k_range:
        # Calcul des distances moyennes pour chaque k
        model = NearestNeighbors(n_neighbors=k)
        model.fit(X_encoded)
        distances, _ = model.kneighbors(X_encoded)
        avg_distances.append(np.mean(distances))

        # Calcul du score de silhouette
        # Nous utilisons KMeans pour créer des clusters et évaluer la qualité
        kmeans = KMeans(n_clusters=k, random_state=42)
        clusters = kmeans.fit_predict(X_encoded)
        if k > 1:  # Le score de silhouette nécessite au moins 2 clusters
            silhouette_scores.append(silhouette_score(X_encoded, clusters))
        else:
            silhouette_scores.append(0)

    return avg_distances, silhouette_scores

In [ ]:
import matplotlib.pyplot as plt

# Définition de la plage de k à tester
k_range = range(2, 21)

# Évaluation des différentes valeurs de k
X_input = X_encoded.drop(columns=['tconst', 'title', 'liste_titles'])
avg_distances, silhouette_scores = evaluate_k(X_input, k_range)

# Création d'une visualisation pour aider à choisir k
plt.figure(figsize=(12, 5))

# Premier graphique : Distance moyenne aux voisins
plt.subplot(1, 2, 1)
plt.plot(k_range, avg_distances, 'bo-')
plt.xlabel('Nombre de voisins (k)')
plt.ylabel('Distance moyenne aux voisins')
plt.title('Distance moyenne en fonction de k')
plt.grid(True)

# Second graphique : Score de silhouette
plt.subplot(1, 2, 2)
plt.plot(k_range, silhouette_scores, 'ro-')
plt.xlabel('Nombre de clusters (k)')
plt.ylabel('Score de silhouette')
plt.title('Score de silhouette en fonction de k')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Trouver k optimal
meilleur_k = k_range[silhouette_scores.index(max(silhouette_scores))]
print(f"Le k optimal est : {meilleur_k}")